In [9]:
!nvidia-smi

Fri Feb 27 17:42:36 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   70C    P0             31W /   70W |     323MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
# --- CLEAN START ---
# Remove everything that can conflict
# !pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece

# --- PYTORCH (CUDA 11.8) ---
# !pip install torch --index-url https://download.pytorch.org/whl/cu118

# --- INSTALL MODERN UNSLOTH (the correct one for Qwen2.5) ---
# !pip install "unsloth==2026.2.1"

# --- INSTALL THE HF STACK COMPATIBLE WITH UNSLOTH 2026.x ---
# !pip install "transformers>=4.57.0" "accelerate>=0.34.1" "peft>=0.18.0" datasets sentencepiece bitsandbytes

# Clean uninstall - remove everything that could conflict
!pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece trl xformers

# Install pinned PyTorch for CUDA 11.8 (good for T4)
!pip install torch --index-url https://download.pytorch.org/whl/cu118

# Install exact Unsloth 2026.2.1 release
!pip install "unsloth==2026.2.1"

# Force-reinstall unsloth_zoo right after to match the 2026.2.1 expectations
!pip install --no-cache-dir --force-reinstall "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git@February-2026"

# Compatible HF ecosystem versions (your original bounds are fine, but explicit helps)
!pip install "transformers>=4.57.0,<4.58.0" "accelerate>=0.34.1" "peft>=0.18.0" "datasets" "sentencepiece" "bitsandbytes" "trl<0.25.0"

##########################################
# 1. Install Unsloth with the latest Colab-specific optimization
# !pip install --force-reinstall --no-cache-dir --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-cache-dir unsloth_zoo
# Force-update both packages together to stay in sync
# !pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --upgrade --no-cache-dir unsloth_zoo
# !pip uninstall -y unsloth unsloth-zoo
# !pip install --upgrade --no-cache-dir "unsloth[colab-new]@git+https://github.com/unslothai/unsloth.git"
# !pip install --upgrade --no-cache-dir unsloth_zoo

# 2. Install essential backends for LoRA and 4-bit quantization
#!pip install --no-deps "xformers<0.0.30" "trl<0.13.0" peft accelerate bitsandbytes

# 3. Install data handling and high-speed download utilities
#!pip install datasets sentencepiece hf_transfer huggingface_hub

# 4. Enable HF_TRANSFER for 10x faster model loading into Colab's local disk
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

Found existing installation: unsloth 2026.2.1
Uninstalling unsloth-2026.2.1:
  Successfully uninstalled unsloth-2026.2.1
Found existing installation: unsloth_zoo 2026.2.1
Uninstalling unsloth_zoo-2026.2.1:
  Successfully uninstalled unsloth_zoo-2026.2.1
Found existing installation: transformers 4.57.6
Uninstalling transformers-4.57.6:
  Successfully uninstalled transformers-4.57.6
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.1
Uninstalling peft-0.18.1:
  Successfully uninstalled peft-0.18.1
Found existing installation: datasets 4.3.0
Uninstalling datasets-4.3.0:
  Successfully uninstalled datasets-4.3.0
Found existing installation: tokenizers 0.22.2
Uninstalling tokenizers-0.22.2:
  Successfully uninstalled tokenizers-0.22.2
Found existing installation: huggingface_hub 0.36.2
Uninstalling huggingface_hub-0.36.2:
  Successfully uninstalled huggingface_hub-0.36.2
Found ex

  Cloning https://github.com/unslothai/unsloth-zoo.git (to revision February-2026) to /tmp/pip-install-prxyvl9k/unsloth-zoo_6b85f811a2d34b47ab6d0329e46f6aec
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-prxyvl9k/unsloth-zoo_6b85f811a2d34b47ab6d0329e46f6aec
  Running command git checkout -q February-2026
  error: pathspec 'February-2026' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q February-2026 did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q February-2026 did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [11]:
import unsloth; print(unsloth.__version__)
import unsloth_zoo; print(unsloth_zoo.__version__ if hasattr(unsloth_zoo, '__version__') else "no __version__ attr")
!pip show unsloth unsloth-zoo | grep -E "Name|Version|Location"

2026.2.1
2026.2.1
Name: unsloth
Version: 2026.2.1
Location: /usr/local/lib/python3.12/dist-packages
Name: unsloth_zoo
Version: 2026.2.1
Location: /usr/local/lib/python3.12/dist-packages


In [12]:
from google.colab import drive
import torch
import os
drive.mount('/content/drive')

with open('/content/drive/MyDrive/TMP/HF_TOKEN.txt') as f:
    token = f.read().strip()
    if token:
        os.environ["HF_TOKEN"] = token
# 2. Check if HF_TOKEN is set in the environment
test_token=os.getenv("HF_TOKEN")
if test_token:
    print("HF_TOKEN is set in the environment.")
else:
    print("HF_TOKEN is NOT set in the environment. Please check your token file and try again.")
    raise ValueError("HF_TOKEN not found in environment variables.")

# 2. Detect Hardware for optimization
major_v, _ = torch.cuda.get_device_capability()
IS_A100 = True if major_v >= 8 else False
print(f"🚀 Hardware: {'A100 (using BF16)' if IS_A100 else 'T4 (using FP16)'}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
HF_TOKEN is set in the environment.
🚀 Hardware: T4 (using FP16)


In [17]:
import os

paths = {
    "Script": "/content/drive/MyDrive/scripts/train_scout_unsloth.py",
    "Data": "/content/drive/MyDrive/data/train/detective_finetune.jsonl",
    "OutputFlder": "/content/drive/MyDrive/outputs/detective-qwen-sft",
    "GGUFOutputFlder": "/content/drive/MyDrive/models/detective-qwen-gguf"
}

for name, path in paths.items():
    if os.path.exists(path):
        print(f"✅ {name} found: {path}")
    else:
        print(f"❌ {name} MISSING: {path}")
        raise FileNotFoundError(f"{name} not found at {path}. Please check the path and try again.")

✅ Script found: /content/drive/MyDrive/scripts/train_scout_unsloth.py
✅ Data found: /content/drive/MyDrive/data/train/detective_finetune.jsonl
✅ OutputFlder found: /content/drive/MyDrive/outputs/detective-qwen-sft
✅ GGUFOutputFlder found: /content/drive/MyDrive/models/detective-qwen-gguf


In [18]:
!pip install llama-cpp-python --force-reinstall --no-cache-dir
!ls -la "/content/drive/MyDrive/outputs/detective-qwen-sft"
!cat "/content/drive/MyDrive/outputs/detective-qwen-sft/adapter_config.json"

total 173271
-rw------- 1 root root      1209 Feb 27 16:35 adapter_config.json
-rw------- 1 root root 161533192 Feb 27 16:35 adapter_model.safetensors
-rw------- 1 root root       605 Feb 27 16:35 added_tokens.json
-rw------- 1 root root      2507 Feb 27 16:35 chat_template.jinja
drwx------ 2 root root      4096 Feb 27 16:25 checkpoint-101
drwx------ 2 root root      4096 Feb 27 16:35 checkpoint-120
-rw------- 1 root root   1671853 Feb 27 16:35 merges.txt
-rw------- 1 root root      5252 Feb 27 16:35 README.md
-rw------- 1 root root       496 Feb 27 16:35 special_tokens_map.json
-rw------- 1 root root      4709 Feb 27 16:35 tokenizer_config.json
-rw------- 1 root root  11421896 Feb 27 16:35 tokenizer.json
-rw------- 1 root root   2776833 Feb 27 16:35 vocab.json
{
  "alora_invocation_tokens": null,
  "alpha_pattern": {},
  "arrow_config": null,
  "auto_mapping": {
    "base_model_class": "Qwen2ForCausalLM",
    "parent_library": "transformers.models.qwen2.modeling_qwen2",
    "unsloth_f

In [21]:
import time
import os
# Force memory management fixes before running the script
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"
# Force torch to avoid using the broken dynamo compiler
os.environ["TORCHDYNAMO_DISABLE"] = "1"

script_path = "/content/drive/MyDrive/scripts/export_to_gguf.py"

print("🎬 Initializing gguf converting session...")
start_time = time.time()

# Run the training script
!python {script_path}

end_time = time.time()
elapsed_mins = (end_time - start_time) / 60

print(f"\n" + "="*40)
print(f"🏁 GGUF CONVERSION COMPLETE")
print(f"⏱️ Total Time: {elapsed_mins:.2f} minutes")
print(f"📂 Output Location: /content/drive/MyDrive/outputs/detective-qwen-gguf")
print("="*40)

🎬 Initializing gguf converting session...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
2026-02-27 17:59:50.658567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772215190.679648   62910 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772215190.686547   62910 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772215190.704001   62910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772215190.704026   62910 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the 